In [80]:
import numpy as np
import pandas as pd

In [81]:
pitching = pd.read_csv("../data/pitching_stats_2024.csv")
batting = pd.read_csv("../data/batting_stats_2024.csv")
game_logs = pd.read_csv("../data/game_logs_2024.csv")

In [82]:
pitching_feature_info = pd.read_csv("../data/pitching_stats_reference.csv", index_col=False)
batting_feature_info = pd.read_csv("../data/batting_stats_reference.csv", index_col=False)

In [83]:
### Function fetching the information of the pitching/batting stats abbreviations
def feature_info(feature, type="pitching"):
    if type == "pitching":
        fullname = pitching_feature_info[pitching_feature_info['Abbreviation'] == feature]["Full Name"].iloc[0]
        desc = pitching_feature_info[pitching_feature_info['Abbreviation'] == feature]["Description"].iloc[0]
        print(feature + " : " + fullname + " , " + desc)
    elif type == "batting":
        fullname = batting_feature_info[batting_feature_info['Abbreviation'] == feature]["Full Name"].iloc[0]
        desc = batting_feature_info[batting_feature_info['Abbreviation'] == feature]["Description"].iloc[0]
        print(feature + " : " + fullname + " , " + desc)
        
#Examples
feature_info('IBB', type='pitching')
feature_info('SF', type='batting')

IBB : Intentional Walks , Intentional bases on balls
SF : Sacrifice Flies , Sacrifice fly outs


### Explanation of the Pitching Features

In [84]:
for col in pitching.columns:
    feature_info(col, type='pitching')

W : Wins , Number of games where pitcher was credited with a win
L : Losses , Number of games where pitcher was credited with a loss
WAR : Wins Above Replacement , Overall value compared to a replacement-level player
ERA : Earned Run Average , Average earned runs per 9 innings
G : Games , Total games appeared in
GS : Games Started , Total games started by the pitcher
CG : Complete Games , Games pitched start to finish
ShO : Shutouts , Complete games with 0 runs allowed
SV : Saves , Games successfully closed in a save situation
IP : Innings Pitched , Total innings pitched
TBF : Total Batters Faced , Total plate appearances faced
H : Hits Allowed , Total hits allowed
R : Runs Allowed , All runs allowed
ER : Earned Runs , Runs charged to pitcher (excluding errors)
HR : Home Runs Allowed , Total home runs given up
BB : Walks , Bases on balls allowed
IBB : Intentional Walks , Intentional bases on balls
HBP : Hit By Pitch , Number of batters hit by pitch
WP : Wild Pitches , Pitches too wild 

### Explanation of the Batting Features

In [85]:
for col in batting.columns:
    feature_info(col, type='batting')

G : Games Played , Total games appeared in
AB : At Bats , Official at-bats (excludes walks, HBP, etc.)
PA : Plate Appearances , Total trips to the plate including walks and HBP
H : Hits , Total hits (singles + doubles + triples + HRs)
1B : Singles , Hits that are singles
2B : Doubles , Hits that are doubles
3B : Triples , Hits that are triples
HR : Home Runs , Hits that are home runs
R : Runs , Total runs scored
RBI : Runs Batted In , Total runs batted in
BB : Walks , Base on balls
IBB : Intentional Walks , Intentional base on balls
SO : Strikeouts , Total strikeouts
HBP : Hit By Pitch , Times hit by a pitch
SF : Sacrifice Flies , Sacrifice fly outs
SH : Sacrifice Hits , Sacrifice bunts
GDP : Ground Into Double Play , Grounded into double plays
SB : Stolen Bases , Bases stolen successfully
CS : Caught Stealing , Times caught stealing
AVG : Batting Average , Hits divided by at-bats
OBP : On-base Percentage , Times on base divided by plate appearances
SLG : Slugging Percentage , Total ba